In this notebook we will approach for the first time PyTorch. 
First we see how `torch` can help us to avoid long `numpy` code and then we will see how to use `torch` to build a simple neural network for classification.

# Part 1: From Numpy to PyTorch

**What will you learn in this part**:
- Tensors syntax
- Autograd
- Neural Network modules

Let's get back to last week exercise and migrate it to PyTorch. Luckily, the syntax is almost identical. The main difference is that *arrays* are replaced by *tensors*, and all the `np.*` functions become `torch.*`. For more advanced functionalities, we refer you to the [official documentation][torch_doc].

[torch_doc]: https://pytorch.org/docs/stable/index.html

## Single layer MLP in Numpy

Recall the feedforward neural network with a single hidden layer.

![simple_mlp](./simple_mlp.png)

Below is the Numpy implementation of the activations and the feedforward propagation

In [ ]:
import numpy as np
from typing import Tuple
from numpy.typing import NDArray

def np_sigmoid(t):
    """apply sigmoid function on t."""
    return 1.0 / (1 + np.exp(-t))

def np_grad_sigmoid(t):
    """return the derivative of sigmoid on t."""
    return np_sigmoid(t) * (1 - np_sigmoid(t))

def np_mlp(
    x: NDArray[np.float64], w_1: NDArray[np.float64], w_2: NDArray[np.float64]
) -> Tuple[NDArray[np.float64], NDArray[np.float64], NDArray[np.float64]]:
    """Feed forward propagation on MLP

    Args:
        x (NDArray[np.float_]): Input vector of shape (d_in,)
        w_1 (NDArray[np.float_]): Parameter matrix of first hidden layer, of shape (d_in, d_hid)
        w_2 (NDArray[np.float_]): Parameter vector of output layer, of shape (d_hid,)

    Returns:
        Tuple[NDArray[np.float], NDArray[np.float], NDArray[np.float]]: Three
            arrays `y_hat`, `z_1`, `z_2`, containing repsectively the output and
            the two preactivations.
    """
    z_1 = w_1.T @ x
    x_1 = np_sigmoid(z_1)
    z_2 = w_2.T @ x_1
    y_hat = np_sigmoid(z_2)

    return y_hat, z_1, z_2


And this is the backpropagation with the Mean-squared error loss $\mathcal L (y, \hat y) = \frac{1}{2} \left( y - \hat y \right)^2$:

In [ ]:
def np_mlp_backpropagation(
    y: NDArray[np.int_],
    x: NDArray[np.float64],
    w_2: NDArray[np.float64],
    y_hat: NDArray[np.float64],
    z_1: NDArray[np.float64],
    z_2: NDArray[np.float64],
) -> Tuple[NDArray[np.float64], NDArray[np.float64]]:
    """Do backpropagation and get parameter gradients.

    Args:
        y (NDArray[np.int_]): True label
        x (NDArray[np.float64]): Input data
        w_2 (NDArray[np.float64]): Readout layer parameters
        y_hat (NDArray[np.float64]): MLP output
        z_1 (NDArray[np.float64]): Hidden layer preactivations
        z_2 (NDArray[np.float64]): Readout layer preactivations

    Returns:
        Tuple[NDArray[np.float64], NDArray[np.float64]]: Gradients of w_1 and w_2
    """
    # Feed forward
    _loss = 0.5 * (y - y_hat)**2

    # Backpropogation
    delta_2 = (y_hat - y) * np_grad_sigmoid(z_2)
    x_1 = np_sigmoid(z_1)
    dw_2 = delta_2 * x_1
    delta_1 = delta_2 * w_2* np_grad_sigmoid(z_1)
    dw_1 = np.outer(x, delta_1)

    return dw_1, dw_2

Now, we can compute the MLP output and retrieve the gradients

In [ ]:
x_np = np.array([0.01, 0.02, 0.03, 0.04])
w_1_np = np.random.randn(4, 5)
w_2_np = np.random.randn(5)

y = 1

y_hat_np, z_1, z_2 = np_mlp(x_np, w_1_np, w_2_np)
dw_1_np, dw_2_np = np_mlp_backpropagation(y, x_np, w_2_np, y_hat_np, z_1, z_2)

print(dw_1_np.shape)
print(dw_2_np.shape)

This indeed works, but as soon as we change the neural network architecture we have to change our backpropagation function, and keep track of all the computations that involve each parameter. It is a lot of work which we want to delegate to the machine.
This is what *automatic differentiation* does, and libraries like PyTorch implement it.

## Exercise 1

We can manipulate tensors as we want and, by asking for `require_grad=True`, PyTorch handles automatic differentation!

In [ ]:
import torch

In [ ]:
# EXAMPLE

a = torch.randn(10, 5)
b = torch.ones(5, requires_grad=True)

# Note that c is a scalar
c = torch.log(a @ b).sum()
print("c", c)
print("c.shape:", c.shape)

# We ask to perform backpropagation
c.backward()

print("b.grad:", b.grad)
print("b.grad.shape:", b.grad.shape)


We now convert the previous code to PyTorch. Autograd is responsible of keeping track of each element in the computations, so we only need to implement the forward pass!

In [ ]:
def sigmoid(t) -> torch.FloatTensor:
    """apply sigmoid function on t."""
    #vvvvv YOUR CODE HERE vvvvv#˙
    return 1.0 / (1 + torch.exp(-t))
    #^^^^^^^^^^^^^^^^^^^^^^^^^^#

def mlp(
    x: torch.Tensor, w_1: torch.Tensor, w_2: torch.Tensor
) -> torch.Tensor:
    """Feed forward propagation on MLP

    Args:
        x (torch.Tensor): Input vector of shape (d_in,)
        w_1 (torch.Tensor): Parameter matrix of first hidden layer, of shape (d_in, d_hid)
        w_2 (torch.Tensor): Parameter vector of output layer, of shape (d_hid,)

    Returns:
        torch.Tensor: Network output
    """
    #vvvvv YOUR CODE HERE vvvvv#
    z_1 = w_1.T @ x
    x_1 = sigmoid(z_1)
    z_2 = w_2.T @ x_1
    y_hat = sigmoid(z_2)
    #^^^^^^^^^^^^^^^^^^^^^^^^^^#

    return y_hat


Now, we can verify that the output corresponds to the numpy implementation

In [ ]:
#vvvvv YOUR CODE HERE vvvvv#

# Convert arrays to tensors. Mind that we will ask for parameters gradients!
x = torch.tensor(x_np)
w_1 = torch.tensor(w_1_np, requires_grad=True)
w_2 = torch.tensor(w_2_np, requires_grad=True)

y_hat = mlp(x, w_1, w_2)
loss = 0.5 * (y - y_hat)**2

#Now perform backpropagation
loss.backward()
#^^^^^^^^^^^^^^^^^^^^^^^^^^#

print(np.allclose(w_1.grad.numpy(), dw_1_np))
print(np.allclose(w_2.grad.numpy(), dw_2_np))


## Exercise 2.1

Computing gradients has now got much easier! :grin:

Still, PyTorch provides an even easier interface to build and train neural networks, whose components are in the `torch.nn` module.
The main tool is the `torch.nn.Module` class, from which all neural networks shall inherit. This must implement a `forward` method, and, if needed, declare its parameters in the `__init__` method. 

Let's convert our MLP to a proper Module

In [ ]:
class MLP(torch.nn.Module):
    def __init__(self, dim_in: int, dim_hidden: int) -> None:
        #vvvvv YOUR CODE HERE vvvvv#
        super().__init__()

        self.w_1 = torch.nn.Parameter(
            torch.randn((dim_in, dim_hidden)),
            requires_grad=True,
        )
        self.w_2 = torch.nn.Parameter(
            torch.randn((dim_hidden, 1)),
            requires_grad=True,
        )
        #^^^^^^^^^^^^^^^^^^^^^^^^^^#

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        #vvvvv YOUR CODE HERE vvvvv#
        z_1 = self.w_1.T @ x
        x_1 = z_1.sigmoid()  # Let's use PyTorch built-in sigmoid!
        z_2 = self.w_2.T @ x_1
        return z_2.sigmoid()
        #^^^^^^^^^^^^^^^^^^^^^^^^^^#

Even better, `torch.nn` comes with a lot of layers and functions which are ready to use.

For instance, we have a `torch.sigmoid` function, as well as `torch.nn.Linear` layer and a `torch.nn.MSELoss` loss.

Here is a minimal implementation of our forward and backward pass:

In [ ]:
from torch import nn

class MyMLP(nn.Module):
    def __init__(self, dim_in: int, dim_hidden: int) -> None:
        super().__init__()

        # NOTE: Linear has a `bias` term by default!
        self.linear1 = nn.Linear(dim_in, dim_hidden, bias=False)
        self.linear2 = nn.Linear(dim_hidden, 1, bias=False)

    def forward(self, x):
        x = self.linear1(x).sigmoid()
        return self.linear2(x).sigmoid()

Now initialize your model and compute the gradients with respect to the MSE loss

In [ ]:
DIM_IN = 5
DIM_HIDDEN = 10

x = torch.ones(DIM_IN)
y = torch.tensor([0.1])

#vvvvv YOUR CODE HERE vvvvv#

my_mlp = MyMLP(DIM_IN, DIM_HIDDEN)
print(my_mlp)

loss = nn.MSELoss()

# Backpropagate

loss(y, my_mlp(x)).backward()

#^^^^^^^^^^^^^^^^^^^^^^^^^^#

## Exercise 2.2


Check the sizes of the gradients of each layer and verify that they correspond to what you expect.

In [ ]:
#vvvvv YOUR CODE HERE vvvvv#

# Note that we have multiple ways to retrieve the parameters
#
# This one works for the MyMLP module define above, but different modules will
# have other submodules and parameter names

gw_1 = my_mlp.linear1.get_parameter('weight')
gw_2 = my_mlp.linear2.get_parameter('weight')

print(gw_1.shape)
print(gw_2.shape)

#^^^^^^^^^^^^^^^^^^^^^^^^^^#

## One more thing...

The `nn.Sequential` module stacks the given layer one after the other.
Still, to get more control on the forward, it is better to stick to self-defined module

In [ ]:
sequential_mlp = nn.Sequential(
    nn.Linear(DIM_IN, DIM_HIDDEN, bias=False),
    nn.Sigmoid(),
    nn.Linear(DIM_HIDDEN, 1, bias=False),
    nn.Sigmoid(),
)

print(sequential_mlp)

# Part 2: Hands on MNIST

**What you will learn in the second part**: This lab serves as an introduction to PyTorch. We will learn the different steps required in training a deep learning model with modern libraries, such as PyTorch.

So, which are these steps?

* Preliminaries:
    * load the train and test datasets, `train_dataset` and `test_dataset` (MNIST in our case)
    * turn the datasets into a "dataloaders": `train_dataloader` and `test_dataloader`
    * define your `model` architecture
    * define your `optimizer`, e.g. SGD


* Training: Now we have all the building blocks and we need to make our model "learn". In most cases, the training follows a specific "recipe". Specifically, we feed the `model` the whole `train_dataset` using batches that come from the `train_dataloader`. We repeat this a certain number of times, called `epochs`. Each epoch consists of `batches`. So what do we do for each batch?
    * zero out the optimizer. In essence we prepare the optimizer for the incoming data
    * compute the output of the model $f(\cdot)$ for our current data: $x\mapsto f(x)$
    * compute the loss: $\mathcal{L}(f(x), y)$ where $y$ denotes the ground truth
    * perform the `backpropagation` algorithm which involves computing the gradients and performing the update rule



## Getting the preliminaries out of the way

In [ ]:
# first we load all the necessary libraries
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as T
from torch.utils.data import DataLoader

We now load the datasets. We are going to work with MNIST and our goal is classify digits. This is a popular dataset and PyTorch offers it out-of-the-box, making our life easy! We simply need to call the corresponding method.

In [ ]:
# The data are given as PIL images. We need to convert our data to a type
# that is readable by a Neural Network. Thus, we use the ToTensor() "transform"
transform = T.Compose([
    T.ToTensor(),
    # T.Normalize((0.1307,), (0.3081,))
])

# load the train dataset
train_dataset = torchvision.datasets.MNIST(
    root='./data/',
    train=True,
    download=True,
    transform=transform)

# load the test dataset
test_dataset = torchvision.datasets.MNIST(
    root='./data/',
    train=False,
    download=True,
    transform=transform)

In [ ]:
# define the hyperparameters
BATCH_SIZE = 1024
TEST_BATCH_SIZE = 2048
LEARNING_RATE = 0.01

# find out which device is available
def device_type():
    if torch.cuda.is_available():
        return 'cuda'
    elif torch.backends.mps.is_available():
        return 'mps'
    else:
        return 'cpu'
DEVICE = torch.device(device_type())
print(DEVICE)

However, we cannot use the whole dataset; it is too large for computers to handle. Instead, we perform *stochastic* gradient descent, i.e. we feed the model part of the data called batches. In order to do so, we use Pytorch DataLoaders. 

In [ ]:
# Construct the dataloader for the training dataset.
# Here we shuffle the data to promote stochasticity.
train_dataloader = torch.utils.data.DataLoader(
    dataset=train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=2)


# Construct the dataloader for the testing dataset.
test_dataloader = torch.utils.data.DataLoader(
    dataset=test_dataset,
    batch_size=TEST_BATCH_SIZE,
    shuffle=False,
    num_workers=2)


Now, let's visualize some samples.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Get the first 10 images of the train dataset.
images = next(iter(train_dataloader))[0][:10]
grid = torchvision.utils.make_grid(images, nrow=5, padding=10)

def show(img):
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1,2,0)), interpolation='nearest')

show(grid)

## Exercise 3
Now, we are ready to define our model. We will start with a simple model, a MultiLayer Perceptron (MLP) with 2 layers.

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class Net(nn.Module):
    def __init__(self):
        # define the different modules of the network
        super().__init__()
        # How many features should our model have?
        self.fc1 = nn.Linear(784, 50)

        # How many outputs should our model have?
        self.fc2 = nn.Linear(50, 10)
        # we also define the non-linearity
        self.relu = nn.ReLU()



    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # ***************************************************
        # INSERT YOUR CODE HERE
        # You should (a) transform the input to a size that is readable
        # by the MLP and (b) pass the input x successively
        # through the layers.
        # ***************************************************
        # transform the image to a vector
        batch_size = x.shape[0]
        x = x.view(batch_size, -1)

        # pass the vectored image through the layers
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x


In [ ]:
# initialize the model
model = Net()

# move model to device
model = model.to(DEVICE)

# define the optimizer
optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE)

We now define:
* the `fit` function that performs the training part
* the `predict` function that takes as input the test dataloader and prints the performance metrics (e.g. accuracy)

In [ ]:
def train_epoch(
    model: nn.Module,
    train_dataloader: DataLoader,
    optimizer: torch.optim.Optimizer,
    device: torch.device
    ):
    '''
    This function implements the core components of any Neural Network training regiment.
    In our stochastic setting our code follows a very specific "path". First, we load a single batch and zero the optimizer. 
    Then we perform the forward pass, compute the gradients and perform the backward pass. And ...repeat!
    '''

    running_loss = 0.0
    model = model.to(device)
    model.train()
    for batch_idx, (data, target) in enumerate(train_dataloader):
        # move data and target to device
        data, target = data.to(device), target.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # do the forward pass
        output = model(data)

        # compute the loss
        loss = F.cross_entropy(output, target)

        # compute the gradients
        loss.backward()


        # perform the gradient step
        optimizer.step()

        # print statistics
        running_loss += loss.item()

    return running_loss / len(train_dataloader.dataset)


def fit(
    model: nn.Module,
    train_dataloader: DataLoader,
    optimizer: torch.optim.Optimizer,
    epochs: int,
    device: torch.device):
    '''
    the fit method simply calls the train_epoch() method for a
    specified number of epochs.
    '''

    # keep track of the losses in order to visualize them later
    # Train for numerous epochs:
    losses = []
    for epoch in range(epochs):
        running_loss = train_epoch(
            model=model,
            train_dataloader=train_dataloader,
            optimizer=optimizer,
            device=device
        )
        print(f"Epoch {epoch}: Loss={running_loss}")
        losses.append(running_loss)

    return losses

In [ ]:
def predict(model: nn.Module, test_dataloader: DataLoader, device: torch.device):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_dataloader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = F.cross_entropy(output, target)
            test_loss += loss.item()
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).sum()

    test_loss /= len(test_dataloader.dataset)
    accuracy = 100. * correct / len(test_dataloader.dataset)

    print(f'Test set: Avg. loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_dataloader.dataset)} ({accuracy:.0f}%)')


We perform a "sanity check". Our model is at the moment initialized randomly and we have 10 classes (each class has approximately the same number of samples). This means that we should get random performance -> ~10% accuracy.

In [ ]:
predict(model=model, test_dataloader=test_dataloader, device=DEVICE)

In [ ]:
# train for 20 epochs
losses = fit(
    model=model,
    train_dataloader=train_dataloader,
    optimizer=optimizer,
    epochs=20,
    device=DEVICE
)

Let's visualize the loss progression.

In [ ]:
plt.plot(losses)

plt.xlabel('Epoch')
plt.ylabel("Loss")
plt.title("Loss progression across epochs")

In [ ]:
predict(model=model, test_dataloader=test_dataloader, device=DEVICE)

The results are not very good. There are some major problems. We see from the plot above that the loss keeps dropping and does not "plateau". This indicates that we can run the optimization a few more epochs and improve the performance. Another point is that our learning rate is too slow or the selection of vanilla SGD as our optimizer is not optimal. In the next section we will see that simply changing the optimizer (from SGD to Adam) yields very different results!

## Exercise 4: CNN

## Evaluated Part

Notice that the MLP does not take into account the nature of images: close pixels convey local information that is important. Using an MLP, we do not have the notion of the "pixel neighbourhood". Therefore, we neglect important information with an MLP. However, there are models better suited for vision problems, such as Convolutional Neural Networks or CNNs.

With the code structure we have created, we can simply define a CNN and test its performance quickly.

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        # ************ YOUR CODE HERE ************
        # define a CNN with 2 convolutional layers, followed by ReLU and Maxpool each,
        # and a fully connected layer at the end.
        # Hint: you could use nn.Sequential() for the convolutional part



    def forward(self, x):
        # ************ YOUR CODE HERE ************


In [ ]:
# initialize model
cnn = CNN().to(DEVICE)

# define the optimizer. Use Adam
optimizer = optim.Adam(cnn.parameters(), lr=LEARNING_RATE)

# train the CNN
losses = fit(
    model=cnn,
    train_dataloader=train_dataloader,
    optimizer=optimizer,
    epochs=15,
    device=DEVICE
)

In [ ]:
# How does the CNN perform compared to the MLP?
predict(model=cnn, test_dataloader=test_dataloader, device=DEVICE)

In [ ]:
plt.plot(losses)

plt.xlabel('Epoch')
plt.ylabel("Loss")
plt.yscale('log')
plt.title("Loss progression across epochs")

# Part 3: To go further: play with CIFAR10

MNIST is a fairly simple dataset. What happens in more challenging datasets? Try to train a network on CIFAR10 dataset and see how it performs. You can use the same code as above, but you need to change the model architecture.